# Load the Libraries

In [ ]:
import os
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
# np.random.seed(9630)
# tf.random.set_seed(9630)

print(tf.__version__)

2.7.0


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Load data

In [ ]:
size_input = 784
size_hidden = [128, 64]
size_output = 10
number_of_train_examples = 60000
number_of_test_examples = 10000

In [ ]:
# load datasets
mnist = tf.keras.datasets.mnist
fashion_mnist = tf.keras.datasets.fashion_mnist

In [ ]:
def mnist_normalize():
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
  X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return (X_train, y_train), (X_test, y_test)

def fashion_normalize():
  (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
  X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
  X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return (X_train, y_train), (X_test, y_test)

In [ ]:
# Split dataset into batches
# (X_train, y_train), (X_test, y_test) = mnist_normalize()
(X_train, y_train), (X_test, y_test) = fashion_normalize()

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
# # Displaying images and labels of the batch

# plt.figure(figsize = (10,10))
# for i in range(25):
#   plt.subplot(5, 5, i+1)
#   plt.xticks([])
#   plt.yticks([])
#   plt.grid(False)
#   plt.imshow(X_train[i], cmap = plt.cm.binary)
#   plt.xlabel(y_train[i].item())

# Not available after reshape from (60000,28,28) to (60000,784)

# Build MLP Model


In [ ]:
# Define class to build MLP model
class MLP(object):
  def __init__(self, size_input, size_hidden, size_output, device = None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer (2)
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. 
    If None, the device will automatically decide during Eager Execution ('gpu')
    """

    self.size_input, self.size_hidden, self.size_output, self.device =\
    size_input, size_hidden, size_output, device

    # Initialize weights between input layer and hidden layer 1
    self.W0 = tf.Variable(tf.random.normal([self.size_input, self.size_input]))
    # Initialize biases for hidden layer 1
    self.b0 = tf.Variable(tf.random.normal([1, self.size_input]))
    # Initialize weights between input layer and hidden layer 1
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden[0]]))
    # Initialize biases for hidden layer 1
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden[0]]))
    # Initialize weights between hidden layer 1 and hidden layer 2
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden[0], self.size_hidden[1]]))
    # Initialize biases for hidden layer 2
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden[1]]))
    # Initialize weights between hidden layer 2 and output layer
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden[1], self.size_output]))
    # Initialize biases for output layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))

    # Define variables to be updated during backpropagation
    self.variables = [self.W0, self.W1, self.W2, self.W3, self.b0, self.b1, self.b2, self.b3]


  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """

    if self.device is not None:
      with tf.device('gpu:0' if self.device == 'gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
    return self.y

  def loss(self, y_pred, y_true):
    """
    y_pred: Tensor of shape (batch_size, size_output)
    y_true: Tensor of shape (batch_size, size_output)
    """

    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)

  def backward(self, X_train, y_train):
    """
    backward pass
    """

    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))

  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """

    # Cast X to float 32
    X_tf = tf.cast(X, dtype=tf.float32)
    # Remember to normalize your dataset before moving forward

    # Compute values in input layer
    what0 = tf.matmul(X_tf, self.W0) + self.b0
    hhat0 = tf.nn.relu(what0)

    # Compute values in hidden layer 1
    what1 = tf.matmul(what0, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)

    # Compute values in hidden layer 2
    what2 = tf.matmul(hhat1, self.W2) + self.b2
    hhat2 = tf.nn.relu(what2)

    # Compute output
    output = tf.matmul(hhat2, self.W3) + self.b3
    output = tf.nn.softmax(output)

    # Now consider two things
    # First, look at inbuild loss functions if they work with softmax and then change this
    # Second, add tf.Softmax(output) and then return this variable
    return output

# Train Model

In [ ]:
# Set number of epochs
NUM_EPOCHS = 10

In [ ]:
def compute_acc(y_pred, y_true):
  maxid = lambda x: np.argmax(x)
  y_pred_max = np.array([maxid(i) for i in y_pred])
  y_true_max = np.array([maxid(j) for j in y_true])
  acc = sum(y_pred_max == y_true_max) / len(y_pred_max)
  return acc

In [ ]:
def compute_std(y_pred):
  y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
  std_dev = np.std(y_pred_tf)
  std_err = std_dev / np.sqrt(len(y_pred_tf))
  variance = std_dev**2
  return std_err, variance

In [ ]:
# Initialize model using GPU
mlp_on_gpu = MLP(size_input, size_hidden, size_output, device = 'gpu')
time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  acc = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(9630)).batch(40)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total = loss_total + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
    acc = acc + compute_acc(preds, outputs)
  print('Number of Epoch: {} - Average MSE: {}' .format(epoch + 1, np.sum(loss_total) / X_train.shape[0]))
  print('                   - Accuracy: {}' .format(acc/len(train_ds)))
 
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
# For per epoch_time
# epoch_time = Total time / Number of epochs

Number of Epoch: 1 - Average MSE: 0.18008072916666668
                   - Accuracy: 0.09958333333333315
Number of Epoch: 2 - Average MSE: 0.17982513020833332
                   - Accuracy: 0.10086666666666663
Number of Epoch: 3 - Average MSE: 0.17985094401041668
                   - Accuracy: 0.10073333333333322
Number of Epoch: 4 - Average MSE: 0.17990478515625
                   - Accuracy: 0.10046666666666657
Number of Epoch: 5 - Average MSE: 0.17992184244791667
                   - Accuracy: 0.1003833333333332
Number of Epoch: 6 - Average MSE: 0.17993863932291668
                   - Accuracy: 0.10029999999999988
Number of Epoch: 7 - Average MSE: 0.17942120768229167
                   - Accuracy: 0.10288333333333331
Number of Epoch: 8 - Average MSE: 0.17997537434895833
                   - Accuracy: 0.10011666666666655
Number of Epoch: 9 - Average MSE: 0.179975390625
                   - Accuracy: 0.1001166666666666
Number of Epoch: 10 - Average MSE: 0.179975390625
               

# One Step Inference

In [ ]:
test_loss_total = tf.Variable(0, dtype=tf.float32)

# test_loss_total = 0.0
test_acc = 0
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  # b = mlp_on_gpu.loss(preds, outputs)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
  test_acc = test_acc + compute_acc(preds, outputs)
  err, var = compute_std(preds)
  
# a = (test_loss_total.numpy() / X_train.shape[0])
# print(X_train.shape[0])
# print(test_loss_total.numpy())
# print(b)
print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_test.shape[0]))
print('Accuracy: {}' .format(test_acc/len(test_ds)))
print('Standard Error: {}' .format(err))
print('Variance: {}' .format(var))

Test MSE: 0.1800
Accuracy: 0.1002
Standard Error: 0.14999999105930328
Variance: 0.08999998927116426


# 10 Trials - Fashion MNIST

In [ ]:
mse_fashion = []
accuracy_fashion = []
err_fashion = []
var_fashion = []
for i in range(10):
  test_loss_total = tf.zeros([1,1], dtype=tf.float32)
  test_acc = 0
  test_err = 0
  test_var = 0
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(25).batch(40)
  for inputs, outputs in test_ds:
    preds = mlp_on_gpu.forward(inputs)
    test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
    test_acc = test_acc + compute_acc(preds, outputs)
    test_err = test_err + compute_std(preds)[0]
    test_var = test_var + compute_std(preds)[1]
  mse_fashion.append(np.sum(test_loss_total) / X_test.shape[0])
  accuracy_fashion.append(test_acc/len(test_ds))
  err_fashion.append(test_err/len(test_ds))
  var_fashion.append(test_var/len(test_ds))
  print('Number of Trial: {} - Average MSE: {}' .format(i + 1, np.sum(test_loss_total) / X_test.shape[0]))
  print('                   - Accuracy: {}' .format(test_acc/len(test_ds)))
  print('                   - Standard Error: {}' .format(test_err/len(test_ds)))
  print('                   - Variance: {}' .format(test_var/len(test_ds)))
 
print(mse_fashion)
print(accuracy_fashion)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

In [ ]:
# # print(np.mean(accuracy_fashion))
# # print(np.var(accuracy_fashion))
# plt.plot(accuracy_fashion, label = 'mean:0.4849, var:0.00001')
# plt.legend(loc='upper right', bbox_to_anchor = (1,1.25))
# plt.xlabel('Accuracy')
# plt.ylabel('Count')
# plt.title('Baseline - Fashion MNIST')
# plt.show()

# 10 Trials - MNIST

In [ ]:
mse_mnist = []
accuracy_mnist = []
err_mnist = []
var_mnist = []
for i in range(10):
  test_loss_total = tf.zeros([1,1], dtype=tf.float32)
  test_acc = 0
  test_err = 0
  test_var = 0
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(25).batch(40)
  for inputs, outputs in test_ds:
    preds = mlp_on_gpu.forward(inputs)
    test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
    test_acc = test_acc + compute_acc(preds, outputs)
    test_err = test_err + compute_std(preds)[0]
    test_var = test_var + compute_std(preds)[1]
  mse_mnist.append(np.sum(test_loss_total) / X_test.shape[0])
  accuracy_mnist.append(test_acc/len(test_ds))
  err_mnist.append(test_err / len(test_ds))
  var_mnist.append(test_var/len(test_ds))
  print('Number of Trial: {} - Average MSE: {}' .format(i + 1, np.sum(test_loss_total) / X_test.shape[0]))
  print('                   - Accuracy: {}' .format(test_acc/len(test_ds)))
  print('                   - Standard Error: {}' .format(test_err/len(test_ds)))
  print('                   - Variance: {}' .format(test_var/len(test_ds)))
print(mse_mnist)
print(accuracy_mnist)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Trial: 1 - Average MSE: 0.092647802734375
                   - Accuracy: 0.5355000000000001
Number of Trial: 2 - Average MSE: 0.09240828247070312
                   - Accuracy: 0.5367999999999997
Number of Trial: 3 - Average MSE: 0.09261939086914063
                   - Accuracy: 0.5355999999999999
Number of Trial: 4 - Average MSE: 0.09212945556640625
                   - Accuracy: 0.5385000000000001
Number of Trial: 5 - Average MSE: 0.092047216796875
                   - Accuracy: 0.5382
Number of Trial: 6 - Average MSE: 0.0917481689453125
                   - Accuracy: 0.5400000000000001
Number of Trial: 7 - Average MSE: 0.090960986328125
                   - Accuracy: 0.5439999999999998
Number of Trial: 8 - Average MSE: 0.09108131103515625
                   - Accuracy: 0.5434000000000001
Number of Trial: 9 - Average MSE: 0.09114312744140625
                   - Accuracy: 0.543
Number of Trial: 10 - Average MSE: 0.09128525390625
                   - Accuracy: 0.54229999999

In [ ]:
# # print(np.mean(accuracy_mnist))
# # print(np.var(accuracy_mnist))
# plt.plot(accuracy_mnist, label = 'mean:0.54, var:0.0000096')
# plt.legend(loc='upper right', bbox_to_anchor = (1,1.25))
# plt.xlabel('Accuracy')
# plt.ylabel('Count')
# plt.title('Baseline - MNIST')
# plt.show()